# IMPORTS

In [1]:
import os
import sys
import datetime
from pathlib import Path
import re
from functools import partial

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon
import geopy
import rasterio
import xlrd
import importlib
import gdal
import ogr
from geocube.api.core import make_geocube
from geocube.rasterize import rasterize_points_griddata, rasterize_points_radial
import ee

import json
import geojson

In [2]:
ee.Initialize()

In [3]:
rootFol = os.path.dirname(os.getcwd())
dataFol = os.path.join(rootFol,"data")
opFol = os.path.join(rootFol,"outputs","final")
libFol = os.path.join(rootFol,"lib")
sys.path.append(libFol)
import gw_utils as gwmod

In [22]:
importlib.reload(gw_utils)

<module 'gw_utils' from 'C:\\Users\\Craig D\\Code\\atree\\lib\\gw_utils.py'>

# Tejasvi CGWB layer

## Explore

In [4]:
# path = os.path.join(rootFol,"data","groundwater","cgwb_stationwise_historical","CGWB_data_wide.csv")
path = os.path.join(opFol,"csv","KA_processed_wDiff.csv")
metacols = ["SNO","STATE","DISTRICT","SITE_TYPE","WLCODE","LON","LAT"]

In [7]:
gwObj = gwmod.WellDataObj(path,metacols)
gwObj.gdf.head()

,SNO,STATE,DISTRICT,SITE_TYPE,WLCODE,LON,LAT,geometry,rech-96,rech-97,...,disc-06,disc-07,disc-08,disc-09,disc-10,disc-11,disc-12,disc-13,disc-14,disc-15
0,1816,KA,Bagalkot,Bore Well,W21196,75.368056,16.162500,POINT (75.36806 16.16250),NaN,NaN,...,NaN,0.09,NaN,1.25,2.76,-0.87,1.11,-12.78,0.77,2.70
1,1817,KA,Bagalkot,Dug Well,W05482,75.787500,16.050000,POINT (75.78750 16.05000),1.38,0.57,...,2.30,4.33,5.11,0.30,0.30,1.03,0.16,2.32,0.20,0.76
2,1818,KA,Bagalkot,Dug Well,W05224,75.677778,15.916667,POINT (75.67778 15.91667),-0.26,6.75,...,5.37,0.07,0.40,0.10,0.25,1.83,2.15,1.67,2.65,7.00
3,1819,KA,Bagalkot,Bore Well,W05694,75.510278,15.916111,POINT (75.51028 15.91611),NaN,NaN,...,1.64,-2.58,0.30,2.00,0.18,1.80,NaN,7.09,5.28,NaN
4,1820,KA,Bagalkot,Dug Well,W05695,75.505556,15.900000,POINT (75.50556 15.90000),3.44,1.15,...,0.00,-3.36,-0.65,2.40,-0.42,1.61,0.00,0.00,0.00,0.00


## Save to raster

In [8]:
print(gwObj.dataCols)

['rech-96', 'rech-97', 'rech-98', 'rech-99', 'rech-00', 'rech-01', 'rech-02', 'rech-03', 'rech-04', 'rech-05', 'rech-06', 'rech-07', 'rech-08', 'rech-09', 'rech-10', 'rech-11', 'rech-12', 'rech-13', 'rech-14', 'rech-15', 'rech-16', 'disc-96', 'disc-97', 'disc-98', 'disc-99', 'disc-00', 'disc-01', 'disc-02', 'disc-03', 'disc-04', 'disc-05', 'disc-06', 'disc-07', 'disc-08', 'disc-09', 'disc-10', 'disc-11', 'disc-12', 'disc-13', 'disc-14', 'disc-15']


In [9]:
import geojson
with open(os.path.join(opFol,"shapefiles","KA_boundary.geojson")) as f:
    geom = geojson.load(f)
    fea = geom["features"][0]['geometry']
    state_boundary = json.dumps(fea)

In [ ]:
col = gwObj.dataCols[0]
# first run of this, with entire gdf gave memory error, shows steps involved in rasterizing layer
cube = make_geocube(vector_data=gwObj.gdf.loc[:,[col,"geometry"]], #.dropna(how='any')
                    resolution=(-0.001, 0.001),
                   interpolate_na_method='linear',
                   geom=state_boundary,
                   rasterize_function=partial(rasterize_points_griddata, method="linear"))

In [22]:
tifPath = os.path.join(opFol,"tif","KA_consolidated_preprocessed_" + col + ".tif")
print(tifPath)
cube[col].rio.to_raster(tifPath)

C:\Users\Craig D\Code\atree\outputs\final\tif\KA_consolidated_preprocessed_rech-96.tif


# CGWB Dataset

## Preprocess

In [332]:
importlib.reload(gw_utils)

<module 'utils' from 'C:\\Users\\Craig D\\Code\\atree\\models\\utils.py'>

<font color='blue'>make Well Data Object</font>
- this makes a df and gdf

In [21]:
cgwbObj = gw_utils.WellDataObj(path,metacols)  # 'KA'
print(cgwbObj.df.shape)
cgwbObj.gdf.head(2)

(2529, 13)


,STATE,DISTRICT,STATION,Latitude,Longtitude,Station Type,May 2020,Jun 2020,Jul 2020,Aug 2020,Sep 2020,Oct 2020,geometry
0,KARNATAKA,DHARWAD,Rayapura A,15.408333,75.061667,Manual,-,-,-,-,-,-,POINT (75.06167 15.40833)
1,KARNATAKA,MANDYA,Haralahalli,12.483333,76.683333,Manual,-,-,-,-,-,-,POINT (76.68333 12.48333)


<font color='blue'>Select only single state</font>

In [22]:
# List of states in dataset
print(cgwbObj.df.STATE.unique().tolist())

['KARNATAKA']


In [23]:
cgwbdf = cgwbObj.subset_gdf('KARNATAKA')
cgwbdf.shape

(2529, 13)

<font color='blue'>Remove Duplicates (entire row)</font>

In [24]:
cgwbdfU = cgwbObj.remove_dups()
print("number of duplicates found:",cgwbObj.num_dups)

number of duplicates found: 0


<font color='blue'>Remove Null Data Rows</font>

In [25]:
cgwbdfNN = cgwbObj.remove_nulls()
print("number of nulls found:",cgwbObj.num_nulls)

number of nulls found: 0


<font color='blue'> Drop Duplicate geometries</font>
- perhaps instead of dropping geom, do a fuzzy correction?

In [26]:
cgwbgdfUG = cgwbObj.remove_dup_geoms()
print("number of duplicate geometries found:",cgwbObj.num_geom_dups)

sample = cgwbObj.get_sample_dup_geoms()
sample

number of duplicate geometries found: 147


,Latitude,Longtitude,May 2020,Jun 2020,Jul 2020,Aug 2020,Sep 2020,Oct 2020,geometry
0,15.408333,75.061667,-,-,-,-,-,-,POINT (75.06167 15.40833)
585,15.408333,75.061667,-,-,-,-,-,-,POINT (75.06167 15.40833)
1446,15.408333,75.061667,-,-,-,-,-,-,POINT (75.06167 15.40833)


<font color = red>Problem 1: Duplicate Geometries (i.e. same lat, long) with different data on water depths</font>
<font color = red><br> - this affects 147 wells of 2529, in Karnataka (CGWB) dataset, 6% of the state dataset</font>

<font color='blue'> Buffer points in dataset</font>

In [27]:
cgwbgdf_proj = cgwbObj.buffer_geoms(5)
print(cgwbgdf_proj.geometry)

0       POLYGON ((8355831.512 1736304.482, 8355831.488...
1       POLYGON ((8536354.619 1400764.880, 8536354.595...
2       POLYGON ((8307902.286 1829248.001, 8307902.262...
3       POLYGON ((8617061.249 1390315.358, 8617061.225...
4       POLYGON ((8365726.578 1456400.244, 8365726.554...
                              ...                        
2524    POLYGON ((8618916.575 2062306.141, 8618916.551...
2525    POLYGON ((8544518.049 1816853.372, 8544518.025...
2526    POLYGON ((8429487.907 1490945.521, 8429487.883...
2527    POLYGON ((8559948.167 1986175.829, 8559948.143...
2528    POLYGON ((8516719.097 1823484.469, 8516719.073...
Name: buffer, Length: 2382, dtype: geometry


## Plot dataset

In [29]:
datacols = [elem for elem in list(cgwbgdfUG.columns) if elem not in metacols]
# print("datacols are of the format 'May-96', 'Aug-96', 'Nov-96', 'Jan-97',..., 'Jan-17'")
print(datacols)

['May 2020', 'Jun 2020', 'Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'geometry']


In [120]:
# cgwbgdfUG.plot(figsize=(15,15),column=datacols[0],vmin=5,vmax=20,legend=True)

## Save to shapefile

In [30]:
cgwbgdfUG.to_file(os.path.join(dataFol,"groundwater","all_shapefiles","cgwbgdfUG_KA.shp"))

G:\Users\Craig\miniconda3\envs\geo_env\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  """Entry point for launching an IPython kernel.


# INDIA-WRIS Shapefile

## Preprocess

In [322]:
importlib.reload(gw_utils)

<module 'utils' from 'C:\\Users\\Craig D\\Code\\atree\\models\\utils.py'>

In [35]:
# iwStnFol = os.path.join()
path = os.path.join(dataFol,"groundwater","all_shapefiles","indiawris_gw_station.zip")
zippath = "zip://"+path
metacols = ['state_name','site_name','agency_nam','latitude','longitude','manual_or_','gwquality']

<font color='blue'>make Well Data Object</font>
- this makes a df and gdf

In [36]:
iwObj = gw_utils.WellDataObj(zippath,metacols)
print(iwObj.df.shape)
iwObj.df.head(2)

(40979, 8)


,state_name,site_name,agency_nam,latitude,longitude,manual_or_,gwquality,geometry
0,WEST BENGAL,KHADINAN STACH YARD P.W.D.,"DWRID, Govt. of West Bengal",22.472622,87.971709,None,None,POINT (87.97171 22.47262)
1,WEST BENGAL,Khejurgechia Bhaslageria Primary School,"DWRID, Govt. of West Bengal",21.939392,87.685326,None,None,POINT (87.68533 21.93939)


<font color='blue'>Select only single state</font>

In [37]:
# List of states in dataset
print(iwObj.df.state_name.unique().tolist())

['WEST BENGAL', 'BIHAR', None, 'Bihar', 'AP', 'Tamil Nadu', 'Kerala', 'TELANGANA', '-', 'Pondicherry', 'Telangana', 'Andaman & Nicobar', 'Karnataka', 'Jammu & Kashmir', 'Andhra Pradesh', 'Maharashtra', 'Chhattisgarh', 'Madhya Pradesh', 'Goa', 'CHHATTISGARH', 'Uttar Pradesh', 'Odisha', 'West Bengal', 'Jharkhand', 'Dadra & Nagar Haveli', 'Gujarat', 'Daman & Diu', 'Rajasthan', 'Tripura', 'Manipur', 'Assam', 'Meghalaya', 'Nagaland', 'Punjab', 'Arunachal Pradesh', 'Haryana', 'Delhi', 'Uttarakhand', 'Himachal Pradesh', 'Chandigarh']


In [38]:
iwdf = iwObj.subset_gdf('Karnataka')
iwdf.shape

(2207, 8)

<font color='blue'>Remove Duplicates (entire row)</font>

In [39]:
iwdfU = iwObj.remove_dups()
print("number of duplicates found:",iwObj.num_dups)

number of duplicates found: 0


<font color='blue'>Remove Null Data Rows</font>

In [40]:
iwdfNN = iwObj.remove_nulls()
print("number of nulls found:",iwObj.num_nulls)

number of nulls found: 0


<font color='blue'> Drop Duplicate geometries</font>
- perhaps instead of dropping geom, do a fuzzy correction?

In [41]:
iwgdfUG = iwObj.remove_dup_geoms()
print("number of duplicate geometries found:",iwObj.num_geom_dups)

sample = iwObj.get_sample_dup_geoms()
sample

number of duplicate geometries found: 124


,latitude,longitude,geometry
8598,13.8125,77.266667,POINT (77.26667 13.81250)
8600,13.8125,77.266667,POINT (77.26667 13.81250)


<font color = red>Problem 1: Duplicate Geometries (i.e. same lat, long) with different data on water depths</font>
<font color = red><br> - this affects 124 wells of 2207, in Karnataka (WRIS) dataset, 5.6% of the state dataset</font>

<font color='blue'> Buffer points in dataset</font>

In [42]:
iwgdf_proj = iwObj.buffer_geoms(5)
# iwgdf_proj.head(2)
print(iwgdf_proj.geometry)

6041     POLYGON ((8531716.307 1304007.453, 8531716.283...
6092     POLYGON ((8530788.644 1307796.153, 8530788.620...
6197     POLYGON ((8532643.969 1316796.140, 8532643.945...
6202     POLYGON ((8528933.320 1316954.058, 8528933.296...
6214     POLYGON ((8529520.839 1318406.937, 8529520.815...
                               ...                        
38075    POLYGON ((8575316.440 1463064.691, 8575316.416...
38076    POLYGON ((8620308.068 1468302.143, 8620308.044...
38089    POLYGON ((8321136.937 1497840.635, 8321136.913...
38092    POLYGON ((8390711.619 1504991.764, 8390711.594...
38094    POLYGON ((8460286.300 1509283.332, 8460286.276...
Name: buffer, Length: 2083, dtype: geometry


## Plot dataset

In [43]:
datacols = [elem for elem in list(iwgdfUG.columns) if elem not in metacols]
# print("datacols are of the format 'May-96', 'Aug-96', 'Nov-96', 'Jan-97',..., 'Jan-17'")
print(datacols)

['geometry']


In [120]:
# cgwbgdfUG.plot(figsize=(15,15),column=datacols[0],vmin=5,vmax=20,legend=True)

# Joins

In [113]:
print("Tej (CGWB) dataset:",tejgdfUG.shape,tejgdf_proj.shape,"\n",
      "IWRIS (CGWB) dataset:",cgwbgdfUG.shape,cgwbgdf_proj.shape,"\n",
      "IWRIS (CGWB) SHAPEFILE:",iwgdfUG.shape,iwgdf_proj.shape)

Tej (CGWB) dataset: (2356, 92) (2356, 93) 
 IWRIS (CGWB) dataset: (2382, 13) (2382, 14) 
 IWRIS (CGWB) SHAPEFILE: (2083, 8) (2083, 9)


In [116]:
join = gpd.sjoin(tejgdf_proj, cgwbgdf_proj, how="inner", op='intersects')  
join.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 2318 entries, 1699 to 27812
Columns: 107 entries, SNO to geometry_right
dtypes: float64(88), geometry(3), int64(2), object(14)
memory usage: 1.9+ MB


In [121]:
# join = gpd.sjoin(tejgdf_proj, iwgdf_proj, how="inner", op='intersects')  # USE THIS TO COMPARE TEJ WITH IW SHAPE  -- LOW MATCH
# join.info()

In [119]:
# join = gpd.sjoin(tejgdfUG, cgwbgdfUG, how="inner", op='intersects')      # USE THIS TO COMPARE POINT-FOR-POINT
# join.info()

<font color = green>In Karnataka, 2318 (97%) of 2382 stations are common between Tejasvi's dataset and the current CGWB locations dataset</font>